In [47]:
from numpy import *
from pylab import *
import string

from datetime import date

import AgilentSCPI
import time
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
import visa
import socket

In [48]:
delayvec=np.arange(1,50,1)
n_aves=4096
t_delay=n_aves*2e-3 #2 ms between pulses
#addtodelay=49#O1 Sample 2
#addtodelay=47 #Sample 1
addtodelay=56 #Sample 3, Orientation1
#addtodelay=47 #Sample 4, Orientation1
fname_root="/Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/"
AMP=[10.0]
Ntrials=1
rm=visa.ResourceManager()
print(rm.list_resources())

8.192
(u'USB0::0x0957::0x1798::MY54100349::INSTR', u'USB0::0x0957::0x2607::MY52202006::INSTR', u'ASRL1::INSTR', u'ASRL2::INSTR', u'ASRL3::INSTR', u'ASRL4::INSTR')


In [49]:
class timedata:

    def __init__(osc, name):
        osc.name = name
        osc.time = np.array    # creates a new empty list
        osc.data = np.array    # creates a new empty list


In [50]:
def readlabdat(fname,dataset):
# read in data from the scope and make a matrix of values from it
    
    
    f = open(fname,'r')
    #header1=f.readline()
    #header2=f.readline()
    tmp1=[]
    tmp2=[]
    
    for line in f:
        columns = line.split(',')
        tmp1.append(float(columns[0]))
        tmp2.append(float(columns[1]))

    dataset.time=np.asarray(tmp1)
    dataset.data=np.asarray(tmp2)
    f.close()
        

In [51]:
def OscRead2NumpyNoNet(osc,channel,npts):                                                                                  
        del osc.timeout                                                                                                    
#        osc.write(':WAVeform:POINts:MODE RAW')                                                                            
        osc.write(':WAVeform:FORMat ASCII')                                                                                
        cmd = ":WAVeform:SOURce CHANnel"+str(channel)                                                                     
        osc.write(cmd)                                                                                                    
                                                                                                                           
        cmd = ":WAVeform:POINts "+str(npts)                                                                                
        # print cmd                                                                                                        
        osc.write(cmd)                                                                                                     
                                                                                                                           
        #print "getting preamble"                                                                                           
        c=osc.ask(":WAVeform:PREamble?")
        a=c.encode('ascii','ignore')
        p=a.split(',')
        p2=[float(ip) for ip in p]
        preamble=np.asarray(p2)
#        t0=np.float(junk[5])                                                                                              
#        dt=np.float(junk[4])                                                                                              
#        nt=np.max(np.shape(data))                                                                                         
#        tvec=np.linspace(t0,t0+dt*nt,nt)                                                                                  
        #print "getting data"                                                                                               
        data_bytes = osc.ask(':WAVeform:DATA?')                                                                            
        data1=data_bytes[10:-1]                                                                                            
        values = data1.split(',') 
        val2=[float(iv) for iv in values]
                                                                                                                           
        data=np.asarray(val2)                                                                                            
                                                                                                                           
        return preamble, data                

In [52]:
# read data from oscilloscope and save to file
def saveWave(osc,filename,chs,npts=4096):
    for ch in chs:
        p1,x1 = OscRead2NumpyNoNet(osc,ch,npts)
        if 'data' in locals():
            data = vstack((data,x1))
        else:
            data = x1
            
    #print p1
    t = arange(len(x1))*p1[4] + p1[5]
    data = vstack((t,data)).T
    savetxt(filename, data, fmt="%12.6G", delimiter=',')
    print(str(shape(data))+ " data points saved to "+filename)

Setup Function Generator

In [53]:
#funcgen = AgilentSCPI.SCPI("USB0::0x0957::0x2607::MY52202006::INSTR")
funcgen = rm.open_resource('USB0::0x0957::0x2607::MY52202006::INSTR')
setupFile="STATE_10"
#setupFile="STATE_BASELINE2"
cmd = ":MMEMory:LOAD:STATe 'INT:\\"+setupFile+"'\n"
funcgen.write(cmd)
funcgen.write(":SOUR2:VOLT"+" "+str(AMP[0])+"\n")
funcgen.write(":SOUR1:VOLT"+" 5.0"+"\n")

(18L, <StatusCode.success: 0>)

Setup Oscilloscope

In [55]:
#osc = rm.open_resource("USB0::0x0957::0x1798::MY56040367::INSTR")
osc = rm.open_resource("USB0::0x0957::0x1798::MY54100349::INSTR")
osc.write("*RST\n")
osc.write("*CLS\n")
osc.write(":TRIGger:EDGE:SOURCe EXTernal\n")
osc.write(":TRIGger:EDGE:LEVel 5.0\n")
osc.write(":TRIGger:EDGE:SLOPe POSitive\n")
osc.write(":TIM:REF LEFT\n")
osc.write(":TIM:DEL 30us\n")
osc.write(":TIM:SCAL 0.5us\n")
osc.write(":CHANnel1:SCALe 0.01\n")
osc.write(":ACQuire:TYPE AVERage\n")
osc.write(":ACQuire:COUNt 4096\n")
osc.write(":WAVeform:POINts 4096")
fname=fname_root+"junk.csv"
saveWave(osc,fname,[1])

(3839, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/junk.csv


In [56]:
for itrial in range(Ntrials):
    for ampval in AMP:
        funcgen.write(":SOUR2:VOLT"+" "+str(ampval)+"\n")
        funcgen.write(":OUTP1 ON\n")
        funcgen.write(":OUTP2 ON\n")
        cmd=":TRIG:DEL "+str(0)+"us\n"
        funcgen.write(cmd)
        fname_root_add=fname_root+"Trial"+str(itrial+1)+"/"+str(ampval)+"V/"
        fname=fname_root_add+str("STrans_B4.csv")
        osc.write(":TIM:REF LEFT\n")
        osc.write(":TIM:SCAL 20us\n")
        osc.write(":TIM:DEL 30us\n")
        osc.write("CHANnel2:SCALe 25\n")
        saveWave(osc,fname,[2])
        osc.write(":TIM:SCAL 1.0us\n")
        scaldel=1
        for delay in delayvec:
            #set delay
            cmd=":TRIG:DEL "+str(delay)+"us\n"
            funcgen.write(cmd)
    
            osc.write(":TIM:DEL "+ str(addtodelay+delay)+"us\n")

            #record probe only
            funcgen.write(":OUTP1 ON\n") #probe on
            funcgen.write(":OUTP2 OFF\n") #PUMP off

            time.sleep(t_delay)

            fname=fname_root_add+"pr"+str(int(scaldel*delay))+".csv"
            saveWave(osc,fname,[1])

            #record PUMP only
            osc.write(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
            funcgen.write(":OUTP1 OFF\n") #probe on
            funcgen.write(":OUTP2 ON\n") #PUMP off
            time.sleep(t_delay)

            fname=fname_root_add+"pu"+str(int(scaldel*delay))+".csv"
            saveWave(osc,fname,[1])

            #record PUMP and probe
            osc.write(":TIM:DEL "+ str(addtodelay+delay)+"us\n")
            funcgen.write(":OUTP1 ON\n") #probe on
            funcgen.write(":OUTP2 ON\n") #PUMP off
            time.sleep(t_delay)

            fname=fname_root_add+"pp"+str(int(scaldel*delay))+".csv"
            saveWave(osc,fname,[1])
            
funcgen.write(":OUTP1 OFF\n")
funcgen.write(":OUTP2 OFF\n")

(3839, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/STrans_B4.csv
(2500, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/pr1.csv
(2500, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/pu1.csv
(2500, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/pp1.csv
(2500, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/pr2.csv
(2500, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/pu2.csv
(2500, 2) data points saved to /Users/amalcolm/Lauren/Baseline/Sample3/Sanded/NoEther/Jan10/Orientation2/StoS/Attempt2/Trial1/10.0V/pp2.csv
(2500, 2) data

(13L, <StatusCode.success: 0>)

In [ ]:
#funcgen.close()
#osc.close()